## Introducción
Con este proyecto mi objetivo es desarrollar un modelo que pueda analizar los datos sobre muestras de crudo de tres regiones junto con los parámetros de cada pozo petrolero de la región y con eso predecir la región con el mayor margen de beneficio.

Observaré los datos de las regiones y crearé un modelo a base de los datos de las regiones y los pozos para predecir las potenciales regiones con menor riesgo y mayor beneficio.

### 1 Voy a cargar las librerías y las tablas con la información de los pozos, además de observar la información de las mismas.

In [1]:
# Cargar librerías.

from scipy import stats as st
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import shuffle

In [2]:
# Carga los datos de los DataFrames.

data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
# Mostrar parte del Dataframe 0

data0.head(10)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [4]:
# Mostrar parte del Dataframe 1

data1.head(10)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [5]:
# Mostrar parte del Dataframe 2

data2.head(10)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [6]:
# Ver descripción general 0

data0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [7]:
# Ver descripción general 1

data1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [8]:
# Ver descripción general 2

data2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Intenté hacer gráficas, pero se muere el kernel. Hago primero el concat y luego grafico el eje "y" como producto y no me lo permitió.

In [9]:
# Ver si hay duplicados 0

print(data0[data0.duplicated()])

Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []


In [10]:
# Ver si hay duplicados 1

print(data1[data1.duplicated()])

Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []


In [11]:
# Ver si hay duplicados 2

print(data2[data2.duplicated()])

Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []


In [12]:
# Ver si hay valores nulos 0

data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [13]:
# Ver si hay valores nulos 1

data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [14]:
# Ver si hay valores nulos 2

data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


### 2 Comenzaré a usar los datasets para entrenar los modelos y medir su exactitud.

In [15]:
# Divido los datos del dataset 0 en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25

target = data0['product']
features = data0.drop(['product', 'id'], axis=1)
features_train, features_valid, target_train, target_valid1 = train_test_split(features, target, test_size=0.25, random_state=54321)

In [16]:
# Entrenamiento de modelo y predicciones

model = LinearRegression() # inicializo el constructor de modelos
model.fit(features_train,target_train) # entreno el modelo en el conjunto de entrenamiento
predictions_valid1 = model.predict(features_valid) # obtengo las predicciones del modelo en el conjunto de validación

result1 = mean_squared_error(target_valid1, predictions_valid1) ** 0.5
# calculo la RECM en el conjunto de validación
print("RECM del modelo de regresión lineal en el conjunto de validación:", result1)
print("Volumen medio de reservas predicho", predictions_valid1.mean())

RECM del modelo de regresión lineal en el conjunto de validación: 37.68341093860808
Volumen medio de reservas predicho 92.15921155743655


In [17]:
# Divido los datos del dataset 1 en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25

target = data1['product']
features = data1.drop(['product', 'id'], axis=1)
features_train, features_valid, target_train, target_valid2 = train_test_split(features, target, test_size=0.25, random_state=54321)

In [18]:
# Entrenamiento de modelo y predicciones

model = LinearRegression() # inicializo el constructor de modelos
model.fit(features_train,target_train) # entreno el modelo en el conjunto de entrenamiento
predictions_valid2 = model.predict(features_valid) # obtengo las predicciones del modelo en el conjunto de validación

result2 = mean_squared_error(target_valid2, predictions_valid2) ** 0.5
# calculo la RECM en el conjunto de validación
print("RECM del modelo de regresión lineal en el conjunto de validación:", result2)
print("Volumen medio de reservas predicho", predictions_valid2.mean())

RECM del modelo de regresión lineal en el conjunto de validación: 0.8922819708481473
Volumen medio de reservas predicho 68.44594093153303


In [19]:
# Divido los datos del dataset 2 en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25

target = data2['product']
features = data2.drop(['product', 'id'], axis=1)
features_train, features_valid, target_train, target_valid3 = train_test_split(features, target, test_size=0.25, random_state=54321)

In [20]:
# Entrenamiento de modelo y predicciones

model = LinearRegression() # inicializo el constructor de modelos
model.fit(features_train,target_train) # entreno el modelo en el conjunto de entrenamiento
predictions_valid3 = model.predict(features_valid) # obtengo las predicciones del modelo en el conjunto de validación
  
result3 = mean_squared_error(target_valid3, predictions_valid3) ** 0.5
# calculo la RECM en el conjunto de validación
print("RECM del modelo de regresión lineal en el conjunto de validación:", result3)
print("Volumen medio de reservas predicho", predictions_valid3.mean())

RECM del modelo de regresión lineal en el conjunto de validación: 40.15250541784855
Volumen medio de reservas predicho 94.92229500787454


### 3 Preparar datos para el cálculo de ganancias:

Almacena todos los valores necesarios para los cálculos en variables separadas.

Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). Compara esta cantidad con la cantidad media de reservas en cada región.

Presenta conclusiones sobre cómo preparar el paso para calcular el beneficio.

In [21]:
# Media región 1
data0['product'].describe()

count    100000.000000
mean         92.500000
std          44.288691
min           0.000000
25%          56.497507
50%          91.849972
75%         128.564089
max         185.364347
Name: product, dtype: float64

In [22]:
# Media región 2

data1['product'].describe()

count    100000.000000
mean         68.825000
std          45.944423
min           0.000000
25%          26.953261
50%          57.085625
75%         107.813044
max         137.945408
Name: product, dtype: float64

In [23]:
# Media región 3

data2['product'].describe()

count    100000.000000
mean         95.000000
std          44.749921
min           0.000000
25%          59.450441
50%          94.925613
75%         130.595027
max         190.029838
Name: product, dtype: float64

Según la información cada 200 pozos necesitan una inversión de 100,000,000 de dólares, por pozo se requiere una producción de 500,000 dólares qué es arriba de 111.1 unidades de volumen de reserva produciendo cada unidad de volumen de reserva 4,500 dólares. Por tanto usaré estos datos para encontrar los mejóres 200 pozos de cada región para ver qué región tiene el mayor beneficio.

In [24]:
# 200 pozos = $100,000,000 inversión
# pozo = $500,000 producción
# 200 pozos = 111.1 unidades de volumen de reserva
# 1 unidad de volumen de reserva = $4,500
# riesgo < 2.5%

In [25]:
# Función para calcular el beneficio

def revenue(targ, predicts):
    pred_ord= pd.Series(predicts).sort_values(ascending= False)    
    selected = targ.reset_index(drop = True).iloc[pred_ord[:200].index] 
    return selected.sum()*4500 - 100000000

In [26]:
# Beneficio de la región 0
revenue1= revenue(target_valid1, predictions_valid1)
print("El beneficio de la región 0 es:", revenue1)

El beneficio de la región 0 es: 31786315.95503962


In [27]:
# Beneficio de la región 1
revenue2= revenue(target_valid2, predictions_valid2)
print("El beneficio de la región 1 es:", revenue2)

El beneficio de la región 1 es: 24150866.966815114


In [28]:
# Beneficio de la región 2
revenue3= revenue(target_valid3, predictions_valid3)
print("El beneficio de la región 2 es:", revenue3)

El beneficio de la región 2 es: 24137856.362028047


### 4 Conclusión del beneficio.

Por medio de la función y los datos observo que el beneficio de la región 0 es el mayor. Por lo tanto es la que concluyo que debe usarse para hacer los pozos.

In [30]:
# Costo de desarrollo de 200 pozos
budget = 100000000  # 100 millones de dólares
cost_per_well = budget / 200

# Ingreso por unidad de producto (4500 dólares por mil barriles)
revenue_per_unit = 4500

# Definir el umbral de ganancias para evitar pérdidas
breakeven_volume = cost_per_well / revenue_per_unit

def calculate_profit(targ, tar_pre, n_wells=200):
    # Seleccionar los n_wells con los valores de predicción más altos
    selected_indices = np.argsort(tar_pre)[-n_wells:]
    selected_reserves = targ.iloc[selected_indices]
    
    # Calcular la ganancia
    total_revenue = selected_reserves.sum() * revenue_per_unit
    total_cost = n_wells * cost_per_well
    profit = total_revenue - total_cost
    
    return profit

def bootstrap_profit(targ, tar_pre, n_iterations=1000, n_wells=200, sample_size=500, seed=42):
    np.random.seed(seed)
    profits = []
    for i in range(n_iterations):
        sample_indices = np.random.choice(range(len(targ)), size=sample_size, replace=True)
        y_true_sample = targ.iloc[sample_indices]
        y_pred_sample = tar_pre[sample_indices]
        
        profit = calculate_profit(y_true_sample, y_pred_sample, n_wells)
        profits.append(profit)
    
    profits = pd.Series(profits)
    mean_profit = profits.mean()
    lower_profit = profits.quantile(0.025)
    upper_profit = profits.quantile(0.975)
    loss_risk = (profits < 0).mean() * 100  # Riesgo de pérdidas como porcentaje
    
    return mean_profit, lower_profit, upper_profit, loss_risk

mean_profit_0, lower_profit_0, upper_profit_0, loss_risk_0 = bootstrap_profit(target_valid1, predictions_valid1)
mean_profit_1, lower_profit_1, upper_profit_1, loss_risk_1 = bootstrap_profit(target_valid2, predictions_valid2)
mean_profit_2, lower_profit_2, upper_profit_2, loss_risk_2 = bootstrap_profit(target_valid3, predictions_valid3)
print(f'Región 0 - Ganancia media: {mean_profit_0:.2f} USD, IC 95%: ({lower_profit_0:.2f}, {upper_profit_0:.2f}), Riesgo de pérdidas: {loss_risk_0:.2f}%')
print(f'Región 1 - Ganancia media: {mean_profit_1:.2f} USD, IC 95%: ({lower_profit_1:.2f}, {upper_profit_1:.2f}), Riesgo de pérdidas: {loss_risk_1:.2f}%')
print(f'Región 2 - Ganancia media: {mean_profit_2:.2f} USD, IC 95%: ({lower_profit_2:.2f}, {upper_profit_2:.2f}), Riesgo de pérdidas: {loss_risk_2:.2f}%')

Región 0 - Ganancia media: 3945069.57 USD, IC 95%: (-1094044.19, 8877458.95), Riesgo de pérdidas: 6.20%
Región 1 - Ganancia media: 4107028.58 USD, IC 95%: (252975.77, 8167733.49), Riesgo de pérdidas: 2.20%
Región 2 - Ganancia media: 3970368.71 USD, IC 95%: (-1654287.63, 9329683.47), Riesgo de pérdidas: 7.70%


### 5 Conclusión luego de calcular riesgos y ganancias para cada región:

Tomando en cuenta los cambios después del bootstrap, antes del bootstrap el beneficio más alto de las regiones era de la región 0 con 31,786,315.95, después del bootstrap el beneficio más alto es de la región 1 con 4,107,028.58 y viendo también que el porcentaje de riesgo es el menor con 2.20%, sin lugar a dudas la región 1 sería la elegida al tener mejor beneficio y menor riesgo.

Me parece de gran ayuda el método bootstrap para hacer estas predicciones apoyado con el intervalo de confianza para que la información sea condensada y acercarse cada vez más a lo real.